In [1]:
import pandas as pd
import datetime as dt
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [3]:
ruta ="dataset/BQ08.2025.xls"
df_bq = pd.read_excel(ruta)

WARNING *** file size (79060) not 512 + multiple of sector size (512)


In [4]:
df_bq.isnull().sum()

Número                0
Nombre                0
Tiempo                0
Estado              122
Dispositivos          0
Tipo de Registro      0
dtype: int64

In [5]:
df_bq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Número            314 non-null    int64 
 1   Nombre            314 non-null    object
 2   Tiempo            314 non-null    object
 3   Estado            192 non-null    object
 4   Dispositivos      314 non-null    object
 5   Tipo de Registro  314 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 14.8+ KB


In [6]:
# Manejar por buenas practicas titulos en minuscula y sin espacios en blanco
new_name =[] 
for column in df_bq.columns:
    minus = column.lower()
    replace = minus.replace(" ","_")
    new_name.append(replace)

df_bq.columns = new_name


df_bq.duplicated().sum()


np.int64(0)

In [7]:
#cambia el nombre de la columna de numero a ID del trabajador
df_bq.rename(columns={'número': 'id'}, inplace=True) 

In [8]:
# Reemplaza el espacio no-breaking por un espacio normal
df_bq['tiempo'] = df_bq['tiempo'].str.replace('\xa0', ' ', regex=False)  

# Reemplaza a.m a AM para que datetime lo lea 
df_bq['tiempo'] = df_bq['tiempo'].str.replace('a. m.', 'AM', regex=False, case=False) 

# Reemplaza p.m a PM para que datetime lo lea 
df_bq['tiempo'] = df_bq['tiempo'].str.replace('p. m.', 'PM', regex=False, case=False) 



df_bq

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro
0,79350377,JAIME CAÑON,1/08/2025 8:07:57 AM,Entrada,barranquilla,0
1,79350377,JAIME CAÑON,1/08/2025 5:05:12 PM,Salida,barranquilla,0
2,79350377,JAIME CAÑON,4/08/2025 2:03:52 PM,Entrada,barranquilla,0
3,79350377,JAIME CAÑON,4/08/2025 5:04:48 PM,Salida,barranquilla,0
4,79350377,JAIME CAÑON,5/08/2025 7:58:02 AM,Entrada,barranquilla,0
...,...,...,...,...,...,...
309,104575797,David de Jesus Fernandez Tette,25/08/2025 7:04:49 AM,Entrada,barranquilla,0
310,104575797,David de Jesus Fernandez Tette,25/08/2025 12:13:06 PM,NaN,barranquilla,0
311,104575797,David de Jesus Fernandez Tette,25/08/2025 1:16:57 PM,NaN,barranquilla,0
312,104575797,David de Jesus Fernandez Tette,25/08/2025 4:03:06 PM,Salida,barranquilla,0


In [9]:
df_bq['tiempo'] = pd.to_datetime(df_bq['tiempo'], dayfirst=True, errors='coerce')

# Crear columnas adicionales
df_bq['fecha'] = df_bq['tiempo'].dt.date
df_bq['hora'] = df_bq['tiempo'].dt.time
df_bq['mes'] = df_bq['tiempo'].dt.month


In [10]:
# Extraer la hora (en formato 24 horas)
df_bq['hora_entera'] = df_bq['tiempo'].dt.hour

# Extraer los minutos
df_bq['minutos_enteros'] = df_bq['tiempo'].dt.minute

# Calcular la fracción de la hora (como en Excel)
df_bq['fraccion_dia_hora'] = (df_bq['hora_entera'] + df_bq['minutos_enteros'] / 60) / 24


In [11]:
# Crear una lista de nombres de días de la semana en español para mapear
dias_semana_espanol = ["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]

# Extraer el número del día de la semana (0=lunes, 6=domingo)
#    Lo aplicamos a la parte de la fecha de la columna 'tiempo'
df_bq['dia_Numero'] = df_bq['tiempo'].dt.weekday

# Mapear el número a su nombre correspondiente en español
df_bq['dia'] = df_bq['dia_Numero'].map(lambda x: dias_semana_espanol[x])


# Busqueda de dias festivos
df_bq['festivo'] = df_bq['fecha'].apply(lambda x: x in co_holidays)

df_bq.info()
df_bq.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 314 non-null    int64         
 1   nombre             314 non-null    object        
 2   tiempo             314 non-null    datetime64[ns]
 3   estado             192 non-null    object        
 4   dispositivos       314 non-null    object        
 5   tipo_de_registro   314 non-null    int64         
 6   fecha              314 non-null    object        
 7   hora               314 non-null    object        
 8   mes                314 non-null    int32         
 9   hora_entera        314 non-null    int32         
 10  minutos_enteros    314 non-null    int32         
 11  fraccion_dia_hora  314 non-null    float64       
 12  dia_Numero         314 non-null    int32         
 13  dia                314 non-null    object        
 14  festivo   

,id,nombre,tiempo,estado,dispositivos,tipo_de_registro,fecha,hora,mes,hora_entera,minutos_enteros,fraccion_dia_hora,dia_Numero,dia,festivo
0,79350377,JAIME CAÑON,2025-08-01 08:07:57,Entrada,barranquilla,0,2025-08-01,08:07:57,8,8,7,0.338194,4,viernes,False
1,79350377,JAIME CAÑON,2025-08-01 17:05:12,Salida,barranquilla,0,2025-08-01,17:05:12,8,17,5,0.711806,4,viernes,False
2,79350377,JAIME CAÑON,2025-08-04 14:03:52,Entrada,barranquilla,0,2025-08-04,14:03:52,8,14,3,0.585417,0,lunes,False
3,79350377,JAIME CAÑON,2025-08-04 17:04:48,Salida,barranquilla,0,2025-08-04,17:04:48,8,17,4,0.711111,0,lunes,False
4,79350377,JAIME CAÑON,2025-08-05 07:58:02,Entrada,barranquilla,0,2025-08-05,07:58:02,8,7,58,0.331944,1,martes,False


In [12]:
df_bq['fecha'] = pd.to_datetime(df_bq['fecha'])

# Agrupar por persona y día, y obtener la primera y última marca
horarios = df_bq.groupby(['nombre', 'fecha'])['tiempo'].agg(['min', 'max']).reset_index()
horarios.rename(columns={'min': 'hora_entrada', 'max': 'hora_salida'}, inplace=True)

horarios['horas_trabajadas'] = (horarios['hora_salida'] - horarios['hora_entrada']).dt.total_seconds() / 3600


In [13]:
horarios['mes'] = horarios['fecha'].dt.month

resumen = horarios.groupby(['nombre', 'mes'])['horas_trabajadas'].sum().reset_index()
resumen.rename(columns={'horas_trabajadas': 'total_horas_efectivas'}, inplace=True)

resumen.head()

,nombre,mes,total_horas_efectivas
0,David de Jesus Fernandez Tette,8,137.048611
1,JAIME CAÑON,8,40.538056
2,KENER DAVID SANTAMARIA,8,145.101667
3,MIGUEL NIEBLES,8,66.977500
4,NEYDER CRESPO,8,151.928056


In [14]:
import pandas as pd
import holidays

def calcular_horas_esperadas(fecha_inicio, fecha_fin, country_code='CO'):
    """
    Calcula las horas laborales esperadas entre dos fechas, excluyendo fines de semana y feriados.

    Args:
        fecha_inicio (str): Fecha de inicio en formato 'YYYY-MM-DD'.
        fecha_fin (str): Fecha de fin en formato 'YYYY-MM-DD'.
        country_code (str): Código del país para los feriados (ej. 'CO' para Colombia).

    Returns:
        float: Total de horas esperadas.
    """
    # Definir horarios de trabajo
    # Lunes a viernes: 9 horas (7 a 16 o 8:30 a 17:30)
    # Sábado: 4 horas (8 a 12 o 9 a 13)
    horas_lunes_viernes = 9
    horas_sabado = 4

    total_horas = 0.0

    # Crear un objeto de la librería holidays para Colombia
    dias_festivos = holidays.CountryHoliday(country_code)

    # Generar un rango de fechas
    fechas = pd.to_datetime(pd.Series(pd.date_range(start=fecha_inicio, end=fecha_fin)))

    for fecha in fechas:
        # Excluir domingos
        if fecha.dayofweek == 6:  # Domingo = 6
            continue

        # Excluir feriados
        if fecha in dias_festivos:
            continue

        # Sumar horas según el día de la semana
        if fecha.dayofweek >= 0 and fecha.dayofweek <= 4: # Lunes a viernes
            total_horas += horas_lunes_viernes
        elif fecha.dayofweek == 5: # Sábado
            total_horas += horas_sabado

    return total_horas

# --- Ejemplo de uso ---
# Para calcular las horas esperadas para agosto hasta el 25
fecha_inicio_agosto = '2025-08-01'
fecha_fin_agosto = '2025-08-25'

horas_esperadas = calcular_horas_esperadas(fecha_inicio_agosto, fecha_fin_agosto)
print(f"Horas laborales esperadas del {fecha_inicio_agosto} al {fecha_fin_agosto}: {horas_esperadas} horas")

Horas laborales esperadas del 2025-08-01 al 2025-08-25: 151.0 horas


In [15]:
resumen['horas_esperadas'] = horas_esperadas
resumen['porcentaje_cumplimiento'] = resumen['total_horas_efectivas'] / resumen['horas_esperadas'] * 100
resumen

,nombre,mes,total_horas_efectivas,horas_esperadas,porcentaje_cumplimiento
0,David de Jesus Fernandez Tette,8,137.048611,151.0,90.760670
1,JAIME CAÑON,8,40.538056,151.0,26.846394
2,KENER DAVID SANTAMARIA,8,145.101667,151.0,96.093819
3,MIGUEL NIEBLES,8,66.977500,151.0,44.355960
4,NEYDER CRESPO,8,151.928056,151.0,100.614606
5,OMAR JOSE CARRILLO BERDEJO,8,42.878333,151.0,28.396247
6,RICARDO ANDRES MEJIA HERNADEZ,8,99.148333,151.0,65.661148


In [16]:
horarios["dia_semana"] = pd.to_datetime(horarios["fecha"]).dt.weekday

# --- 3. Horas esperadas por día ---
horarios["horas_esperadas"] = horarios["dia_semana"].map(
    {0: 9, 1: 9, 2: 9, 3: 9, 4: 9, 5: 4}  # domingo no aparece
)

# --- 4. Detectar errores ---
horarios["trabajo_menos"] = horarios["horas_trabajadas"].round(2) < horarios["horas_esperadas"]
horarios["trabajo_mas"] = horarios["horas_trabajadas"].round(2) > horarios["horas_esperadas"]




horarios = horarios[["nombre","fecha","hora_entrada","hora_salida","horas_trabajadas","horas_esperadas","trabajo_menos","trabajo_mas"]]

In [17]:
with pd.ExcelWriter("entregas/reporte_agosto_barranquilla.xlsx") as writer:
    resumen.to_excel(writer , sheet_name ="resumen",index=False)
    horarios.to_excel(writer , sheet_name= "diario", index =False)
print("Archivo creado correctamente")

Archivo creado correctamente


In [18]:
horarios

,nombre,fecha,hora_entrada,hora_salida,horas_trabajadas,horas_esperadas,trabajo_menos,trabajo_mas
0,David de Jesus Fernandez Tette,2025-08-01,2025-08-01 07:06:24,2025-08-01 17:03:17,9.948056,9,False,True
1,David de Jesus Fernandez Tette,2025-08-02,2025-08-02 08:09:01,2025-08-02 12:42:59,4.566111,4,False,True
2,David de Jesus Fernandez Tette,2025-08-04,2025-08-04 07:39:38,2025-08-04 16:27:23,8.795833,9,True,False
3,David de Jesus Fernandez Tette,2025-08-05,2025-08-05 07:09:01,2025-08-05 13:20:55,6.198333,9,True,False
4,David de Jesus Fernandez Tette,2025-08-06,2025-08-06 07:04:29,2025-08-06 18:26:29,11.366667,9,False,True
...,...,...,...,...,...,...,...,...
99,RICARDO ANDRES MEJIA HERNADEZ,2025-08-21,2025-08-21 08:34:12,2025-08-21 18:10:27,9.604167,9,False,True
100,RICARDO ANDRES MEJIA HERNADEZ,2025-08-22,2025-08-22 08:30:29,2025-08-22 17:33:56,9.057500,9,False,True
101,RICARDO ANDRES MEJIA HERNADEZ,2025-08-23,2025-08-23 08:56:26,2025-08-23 13:06:34,4.168889,4,False,True
102,RICARDO ANDRES MEJIA HERNADEZ,2025-08-25,2025-08-25 07:04:41,2025-08-25 16:08:27,9.062778,9,False,True
